## Step 1: Import the required libraries

In [5]:
import cv2
import numpy as np
from mtcnn import MTCNN
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.layers import Dropout
import datetime
import openpyxl
from tensorflow.keras.models import load_model

## Step 2: Train CNN model using VGG-Face model architecture

In [6]:
#defining data set
dataset_path=r"C:\Users\marey\OneDrive\Documents\Test_22\data"
Train_Data=tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    rescale=1/255.0,
).flow_from_directory(dataset_path,batch_size=16,subset="training",target_size=(224,224),shuffle=False)
class_labels =list(Train_Data.class_indices.keys())

Found 1934 images belonging to 19 classes.


In [7]:
class_labels

['AbdulAziz',
 'Abdulrahman Khalid',
 'Abdulrhman Younis',
 'Afnan',
 'Ahmed',
 'Aishah',
 'Anas',
 'Hala',
 'Hassan',
 'Hazem',
 'Mariyyah',
 'Marwah',
 'Muhammad Alhudari',
 'Muhammed Dhabaab',
 'Omar',
 'Shahad Alnami',
 'Shoog',
 'Snd',
 'Tariq']

- Train The Model

In [4]:
# Create an ImageDataGenerator for data preprocessing and augmentation
data_generator = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# Load the data and split into training and validation sets
data = data_generator.flow_from_directory(dataset_path, target_size=(224, 224), batch_size=32, class_mode='categorical')
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Concatenate the train_data and val_data tuples
train_x = np.concatenate([x for x, y in train_data])
train_y = np.concatenate([y for x, y in train_data])
val_x = np.concatenate([x for x, y in val_data])
val_y = np.concatenate([y for x, y in val_data])

# Define the VGG-Face model architecture
model2 = Sequential()
model2.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D((2, 2), strides=(2, 2)))
model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(19, activation='softmax', kernel_regularizer=regularizers.l2(0.015)))

# Compile the model
model2.compile(optimizer=optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Define a checkpoint to save the model weights after every epoch
checkpoint = ModelCheckpoint('vgg_face_weights_{epoch:02d}.h5', save_weights_only=True)

# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5)

# Train the model on the training data and validate on the validation data
model2.fit(train_x, train_y, epochs=20, steps_per_epoch=len(train_x) // 32, validation_data=(val_x, val_y),
           validation_steps=len(val_x) // 32, callbacks=[checkpoint, early_stop])

Found 1934 images belonging to 19 classes.
Epoch 1/20
47/47 [==============================] - 228s 5s/step - loss: 3.4155 - accuracy: 0.0948 - val_loss: 3.0646 - val_accuracy: 0.1514
Epoch 2/20
47/47 [==============================] - 214s 5s/step - loss: 2.8694 - accuracy: 0.2179 - val_loss: 2.6472 - val_accuracy: 0.2764
Epoch 3/20
47/47 [==============================] - 208s 4s/step - loss: 2.4858 - accuracy: 0.3333 - val_loss: 2.1771 - val_accuracy: 0.4303
Epoch 4/20
47/47 [==============================] - 216s 5s/step - loss: 2.1546 - accuracy: 0.4191 - val_loss: 1.9294 - val_accuracy: 0.4976
Epoch 5/20
47/47 [==============================] - 201s 4s/step - loss: 1.8754 - accuracy: 0.4874 - val_loss: 1.7173 - val_accuracy: 0.5577
Epoch 6/20
47/47 [==============================] - 199s 4s/step - loss: 1.8163 - accuracy: 0.5158 - val_loss: 1.5371 - val_accuracy: 0.6442
Epoch 7/20
47/47 [==============================] - 208s 4s/step - loss: 1.6231 - accuracy: 0.5667 - val_loss: 

In [5]:
# Save the model
model2.save('EVA2_ver5_model.h5')

c:\Users\marey\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Step 3: Make predictions and save attendance information in real-time

In [8]:
import os

# Get the current working directory
root_directory = os.getcwd()

# Create a folder for attendance records
attendance_folder = f"{root_directory}/attendance"
os.makedirs(attendance_folder, exist_ok=True)

# Create a new workbook object (Excel) for attendance records
attendance_workbook = openpyxl.Workbook()
tody_date = str(datetime.datetime.now().date())
attendance_file_name = f"{attendance_folder}/attendance_{tody_date}.xlsx"

# Select the active worksheet
attendance_sheet = attendance_workbook.active

In [9]:
# Load the trained model 
model = load_model(r"C:\Users\marey\OneDrive\Documents\Test_22\EVA2_ver5_model.h5")

# Create a face detection object
detector = MTCNN()

# Define the threshold for recognizing a student
threshold = 0.4

In [4]:
# Add some data to the worksheet
attendance_sheet["A1"] = "Photo"
attendance_sheet["B1"] = "Name"
attendance_sheet["C1"] = "Time"
excel_row = 2
handle_Repetition = {}


# Initialize the video capture object
cap = cv2.VideoCapture(0)

pred_name = ""
face_region = None

# Process the video stream frame by frame
while True:
    # Capture a frame from the video stream
    ret, frame = cap.read()

    # Detect faces in the frame
    detections = detector.detect_faces(frame)

    # Process each detected face
    for detection in detections:
        # Extract the face region from the frame
        x, y, w, h = detection['box']
        face_region = frame[y:y+h, x:x+w]

        # Preprocess the face region
        face_image = cv2.resize(face_region, (224, 224))
        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
        face_image = image.img_to_array(face_image)
        face_image = np.expand_dims(face_image, axis=0)
        face_image = face_image / 255.0

        # Pass the preprocessed face region through the trained model to obtain the predicted class probabilities
        class_probabilities = model.predict(face_image)

        # Identify the person in the frame if the predicted class probability is above the threshold
        max_probability = np.max(class_probabilities)
        if max_probability > threshold:
            class_index = np.argmax(class_probabilities)
            pred_name = class_labels[class_index] 
        else:
            pred_name = "Unknown"

        # Draw a bounding box and label around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, pred_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the processed frame
    cv2.imshow('frame', frame)

    # Exit if the user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the attendance information in an Excel file
if pred_name not in handle_Repetition and face_region is not None:
    # Get the current time
    time = datetime.datetime.now().time()

    # Save the attender's photo in the attendance folder
    attender_photo_path = f'{attendance_folder}/{pred_name}.jpg'
    cv2.imwrite(attender_photo_path, face_region)

    # Write the attendance information in the Excel worksheet
    attendance_sheet["A" + str(excel_row)] = attender_photo_path
    attendance_sheet["B" + str(excel_row)] = pred_name
    attendance_sheet["C" + str(excel_row)] = time

    # Update the number of times the person has been recognized
    handle_Repetition[pred_name] = 1
    excel_row += 1
    attendance_workbook.save(attendance_file_name)

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 153ms/step
